#### General notebook settings

In [1]:
%matplotlib inline

In [2]:
import sys
import os

In [306]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
from scipy.stats import pearsonr
from scipy.stats import bartlett, levene, normaltest, mannwhitneyu
from scipy.stats.mstats import zscore
from scipy.stats import hypergeom
from statsmodels.sandbox.stats.multicomp import multipletests
import statsmodels.graphics.gofplots as sm
import random
import math

In [4]:
DIR = "InOutDirectory/"

## Task 1: KEGG and gene id mapping

Familiarize yourself with the KEGG Rest interface and how to access it with Biopyhton:

http://www.genome.jp/kegg/rest/keggapi.html

http://nbviewer.jupyter.org/github/widdowquinn/notebooks/blob/master/Biopython_KGML_intro.ipynb

### Subtask 1.1 Extract gene lists for all (mouse) KEGG pathways and store them in a suitable Python data structure

Import for the connection with KEGG.

In [7]:
from Bio import SeqIO
from Bio.KEGG.REST import *
from Bio.KEGG.KGML import KGML_parser
from Bio.Graphics.KGML_vis import KGMLCanvas
from Bio.Graphics.ColorSpiral import ColorSpiral

In [8]:
# check the connection to KEGG
print(kegg_info("kegg").read())

kegg             Kyoto Encyclopedia of Genes and Genomes
kegg             Release 79.0+/09-27, Sep 16
                 Kanehisa Laboratories
                 pathway     462,412 entries
                 brite       165,895 entries
                 module      377,109 entries
                 disease       1,657 entries
                 drug         10,364 entries
                 environ         850 entries
                 orthology    20,123 entries
                 genome        4,746 entries
                 genes     20,175,595 entries
                 dgenes       99,485 entries
                 compound     17,748 entries
                 glycan       10,994 entries
                 reaction     10,248 entries
                 rpair         9,661 entries
                 rclass        3,059 entries
                 enzyme        6,791 entries



In [109]:
mouseGenesKEGG_pathways = kegg_list("pathway", "mmu").read()

In [110]:
mouseGenesKEGG_pathwayNames = [line.split("\t") for line in mouseGenesKEGG_pathways.split("\n")]

To get all genes for all pathways of mus musculus, the command kegg_get was used on every pathway ID. A suitable datastructure for storing all data (pathway ID, pathway description and appropriated genes) seems to be a python dictionary with the pathway ID as key. Just the gene names of every gene were stored.

In [157]:
# initialise dictionary
mousePathway = {} 

# loop over all pathways to get all genes for each pathway
for l in mouseGenesKEGG_pathwayNames[:-1]:
    
    # get all genes of a pathway
    temp_genes = kegg_get(l[0]).read()
    
    if "GENE" in temp_genes:
        # get all genes if this pathway contains genes
        temp_genes = temp_genes.replace("COMPOUND", "GENE").split("GENE")[1].split("\n")
        
        # create a list, starting with the pathway description followed by all genes in that pathway
        pathway_info = [l[1]] + [gene.split(";")[0].split(" ")[-1] for gene in temp_genes[:-1]]
    else:
        # else report just the pathway description as info
        pathway_info = [l[1]]
    
    # set the pathway id as key for the dictionary and fill it with the information list (description and genes)
    mousePathway[l[0]] = pathway_info

path:mmu00010
path:mmu00020
path:mmu00030
path:mmu00040
path:mmu00051
path:mmu00052
path:mmu00053
path:mmu00061
path:mmu00062
path:mmu00071
path:mmu00072
path:mmu00100
path:mmu00120
path:mmu00130
path:mmu00140
path:mmu00190
path:mmu00220
path:mmu00230
path:mmu00232
path:mmu00240
path:mmu00250
path:mmu00260
path:mmu00270
path:mmu00280
path:mmu00290
path:mmu00300
path:mmu00310
path:mmu00330
path:mmu00340
path:mmu00350
path:mmu00360
path:mmu00380
path:mmu00400
path:mmu00410
path:mmu00430
path:mmu00450
path:mmu00471
path:mmu00472
path:mmu00480
path:mmu00500
path:mmu00510
path:mmu00511
path:mmu00512
path:mmu00514
path:mmu00520
path:mmu00524
path:mmu00531
path:mmu00532
path:mmu00533
path:mmu00534
path:mmu00561
path:mmu00562
path:mmu00563
path:mmu00564
path:mmu00565
path:mmu00590
path:mmu00591
path:mmu00592
path:mmu00600
path:mmu00601
path:mmu00603
path:mmu00604
path:mmu00620
path:mmu00630
path:mmu00640
path:mmu00650
path:mmu00670
path:mmu00730
path:mmu00740
path:mmu00750
path:mmu00760
path:m

### Subtask 1.2: Save the KEGG gene sets as a gmt file after you made sure they have the proper gene ids with respect to your DE analysis

hints: 

http://biopython.org/wiki/Annotate_Entrez_Gene_IDs

http://www.broadinstitute.org/cancer/software/gsea/wiki/index.php/Data_formats

In [180]:
# initialise outfile
with open(DIR + "CompleteMousePathway.gmt", "w") as out:
    # emptystring for final save
    mouseGmt = ""
    for k in mousePathway:
        mouseGmt += k + "\t" + str(mousePathway[k]).replace("'", "")\
                                                   .replace("]", "")\
                                                   .replace("[", "")\
                                                   .replace(", ", "\t")\
                    + "\n"
                     
    out.write(mouseGmt)

## Task 2: Gene Set Enrichment

### Subtask 2.1: Read in the csv file you produced during the Differential Expression module, extract a gene list (as a python list of gene symbols) from your favorite multiple correction column (and store it in a variable)

In [191]:
len(mousePathway["path:mmu00010"]) -1

66

In [204]:
fullMHTTable = pd.read_csv(DIR + "MHTsandFC.csv", index_col=0)

In [211]:
fullMHTTable.head()

,p_value,sidak_cor_pval,holm-sidak_cor_pval,holm_cor_pval,simes-hochberg_cor_pval,hommel_cor_pval,fdr_bh_cor_pval,fdr_by_cor_pval,fdr_tsbh_cor_pval,fdr_tsbky_cor_pval,log2FC
Igh,8.634098e-08,1.883393e-03,1.720503e-03,1.721984e-03,1.721725e-03,1.517184e-03,9.953373e-07,1.051919e-05,4.384973e-07,4.653523e-07,0.036313
Hcfc2,8.742707e-05,8.517664e-01,7.667686e-01,1.000000e+00,5.000000e-01,4.995801e-01,3.677223e-04,3.886259e-03,1.620006e-04,1.719221e-04,0.019963
Ccdc112,1.072951e-11,2.342685e-07,2.305239e-07,2.305236e-07,2.305129e-07,2.250516e-07,6.657274e-10,7.035715e-09,2.932872e-10,3.112491e-10,0.077491
EG635895,7.292464e-07,1.579628e-02,1.394280e-02,1.404091e-02,1.404091e-02,1.153522e-02,6.165232e-06,6.515702e-05,2.716102e-06,2.882445e-06,0.064943
Srp72,3.539137e-01,1.000000e+00,1.000000e+00,1.000000e+00,5.000000e-01,5.000000e-01,3.899639e-01,1.000000e+00,1.717992e-01,1.823207e-01,0.000477


Two different gene lists are required for further analysis. One with all genes with are in both datasets, all genes which are occuring in the expressed genes and the genes from the KEGG, one with DE genes.

In [234]:
# all genes in KEGG, ignore the description they will automatically be removed in the next step
keggenes = []
for k in mousePathway:
    keggenes.extend(mousePathway[k])

# all genes in expression data
exprgenes = [str(gene) for gene in fullMHTTable.index.values]

# genes in both datasets
commongenes = list(set(keggenes) & set(exprgenes))

# differentially expressed genes
diffexprgenes = [str(gene) for gene in fullMHTTable[fullMHTTable.sidak_cor_pval < 0.05].index.values]
diffexprgenes = set(diffexprgenes) & set(commongenes)

In [244]:
crossDF = pd.DataFrame([[False, False]] * len(commongenes), index=commongenes, columns=["DE", "Pathway"])
crossDF.DE.loc[diffexprgenes] = True

In [347]:
crossDF.head()

,DE,Pathway
Plekhg5,False,False
Nampt,False,False
Yod1,False,False
Rgs14,False,False
Shank3,False,False


### Subtask 2.2: Perform gene set enrichment (Fisher's exact test or an hypergeometric test will do for our purposes) with the KEGG gene sets you extracted in Task 1 (you may want to store the results in a pandas dataframe and write them to csv)

hint:

https://genetrail2.bioinf.uni-sb.de/help?topic=set_level_statistics

In [398]:
signPathways = pd.DataFrame([1.0] * len(mousePathway), index=mousePathway, columns=["p_value"])

In [345]:
# test on one pathway
pathwaygenes = list(set(mousePathway["path:mmu00010"]) & set(commongenes))

curCrossDF = crossDF.copy()
curCrossDF.Pathway.loc[pathwaygenes] = True
ct = pd.crosstab((curCrossDF.DE == True).replace([False, True], ["not DE", "DE"]), 
                 (curCrossDF.Pathway == True).replace([False, True], ["not Pathway", "Pathway"]),
                 margins=True)
n = ct.All.DE
l = ct.Pathway.All
m = ct.All.All
k = ct.Pathway.DE

kprime = (n * l) / m

if kprime >= k:
    print hypergeom.cdf(k, m, l, n)
else:
    print hypergeom.sf(k-1, m, l, n)

0.175265232642


In [399]:
# for all pathways
for key in mousePathway:
    # get all possible pathway genes
    pathwaygenes = list(set(mousePathway[key]) & set(commongenes))
    
    if len(pathwaygenes) == 0:
        continue

    # prepare a cross table for the current pathway 
    curCrossDF = crossDF.copy()
    curCrossDF.Pathway.loc[pathwaygenes] = True
    ct = pd.crosstab((curCrossDF.DE == True).replace([False, True], ["not DE", "DE"]), 
                     (curCrossDF.Pathway == True).replace([False, True], ["not Pathway", "Pathway"]),
                     margins=True)
    # get the variables according to GeneTrail
    n = ct.All.DE
    l = ct.Pathway.All
    m = ct.All.All
    k = ct.Pathway.DE
    kprime = (n * l) / m

    # select the hypergeometric test and store it
    if kprime >= k:
        signPathways.p_value.loc[key] = hypergeom.cdf(k, m, l, n)
    else:
        signPathways.p_value.loc[key] = hypergeom.sf(k-1, m, l, n)

In [400]:
signPathways.to_csv(DIR + "SignificantPathways.csv", na_rep="x")

### Subtask 2.3: Extract a list of significantly (at 0.05 significance) enriched KEGG pathways

In [401]:
res = multipletests(signPathways.p_value.values, alpha=0.05, method="sidak")
res = pd.DataFrame([res[1], res[0]], index =["sidak_cor_pval", "sidak_rej"], columns=signPathways.index).T
signPathways = signPathways.join(res)

In [406]:
signPathways[signPathways.p_value < 0.05]

,p_value,sidak_cor_pval,sidak_rej
path:mmu00052,3.669043e-02,0.999987,False
path:mmu00190,1.199564e-02,0.973868,False
path:mmu00512,3.124239e-02,0.999931,False
path:mmu05202,2.181985e-02,0.998722,False
path:mmu05016,9.387543e-03,0.942065,False
path:mmu05010,1.864683e-02,0.996602,False
path:mmu05012,2.834375e-03,0.57565,False
path:mmu05332,1.826097e-03,0.424195,False
path:mmu04390,1.112213e-02,0.965874,False
path:mmu04962,1.675708e-02,0.993925,False


In [403]:
signPathways[signPathways.sidak_rej == True]

,p_value,sidak_cor_pval,sidak_rej
path:mmu04740,1.342473e-13,4.05362e-11,True


After multiple hypothesis testing with the Sidak method, just one pathway (mmu04740) remains as significant aftergene set enrichment. Without the correction it would have been 24 pathways.

## Task 3: KEGG map visualization

#### hint:

http://nbviewer.jupyter.org/github/widdowquinn/notebooks/blob/master/Biopython_KGML_intro.ipynb

#### remark:

In real life you may want to use the R-based tool pathview: https://bioconductor.org/packages/release/bioc/html/pathview.html (if you insist you can also try to use r2py for using pathview from Python during the practical)

For Python (in addition to the Biopyhton module) https://github.com/idekerlab/py2cytoscape in combination with https://github.com/idekerlab/KEGGscape may be another alternative (in the future)

Generally speaking, it is always a good idea to pay attention also to other pathway databases like Reactome or WikiPathways ...

### Subtask 3.1: Pick some significantly enriched KEGG pathways of your choice from 2.3 and visualize them

### Subtask 3.2: Define a a suitable binary color scheme respresenting the fact whether a gene is significantly expressed or not

hint: 

http://www.rapidtables.com/web/color/RGB_Color.htm

### Subtask 3.3: Visualize the pathway(s) from 3.1 in such a way that the included genes have the corresponding color from 3.2 ( you may need to define a suitable mapping from single genes to what is actually shown in the pathway map...)

### Subtask 3.4: Define a suitable continuous color range representing the log2 fold changes of the all the genes in your data

hint:

http://bsou.io/posts/color-gradients-with-python

### Subtask 3.5: Visualize the pathway(s) from 3.1 in such a way that the included genes have the corresponding color from 3.4